# Double Pendulums
## Introduction
This document introduce the well-known physical double-inverted-pendulum, or the "Acrobot" by [Mark W. Spong's paper](https://ieeexplore.ieee.org/document/341864).

## Acrobot - The physical double inverted pendulum
<img src="figures/Acrobot_Double_Inverted_Pendulum.png" alt="Drawing" style="width: 400px;"/>

The equations of motion in Spong's paper is given by

$$
\begin{align}
&\begin{bmatrix} 
m_1 l_{c1}^2 + m_2(l_1^2 + l_{c2}^2 + 2 l_1 l_{c2} C_{q2}) + I_1 + I_2  
&& 
m_2( l_{c2}^2 + l_1 l_{c2} C_{q2} + I_2)
\\
m_2( l_{c2}^2 + l_1 l_{c2} C_{q2} + I_2)
&&
m_2 l_{c2}^2 + I_2
\end{bmatrix}
\begin{bmatrix}
\ddot{q}_1 \\
\ddot{q}_2
\end{bmatrix}
\\
+
&\begin{bmatrix}
-m_2 l_1 l_{c2} S_{q2} \dot{q}_2^2 - 2 m_1 l_1 l_{c2} S_{q2} \dot{q}_1 \dot{q}_2
\\
m_2 l_1 l_{c2} S_{q2} \dot{q}_1^2
\end{bmatrix}
+
\begin{bmatrix}
(m_1 l_{c1} + m_2 l_1)g C_{q1} + m_2 l_{c2} g C_{q1+q2}
\\
m_2 l_{c2}gC_{q1 + q2}
\end{bmatrix}
=
\begin{bmatrix}
\tau_1 
\\
\tau_2
\end{bmatrix}
\end{align}
$$

In [1]:
# import libraries
from sympy.physics.vector import dynamicsymbols
from sympy import symbols
from sympy import Matrix, cos, sin, diff
from sympy import simplify

# lazy method
# from sympy import *

### Define Arcobot's Symbols

First define the symbols we will use later.

In [2]:
# Time dependent symbols
q1, q2 = dynamicsymbols('q_1 q_2')  # angle
dq1, dq2 = dynamicsymbols('q_1 q_2',1)  # angular velocity
ddq1, ddq2 = dynamicsymbols('q_1 q_2',2)  # angular acceleration

# Scalar symbols
m1, m2, l1, l2, lc1, lc2, I1, I2, t, g = symbols('m_1 m_2 l_1 l_2 l_c1 l_c2 I_1 I_2 t g ')

### Define the displacement and velocity vector of the Acrobot

The displacement vector of Arcobot is given by
$$
\vec{r}_1 = 
\begin{bmatrix} 
l_{c1} C_{q1} \\
l_{c1} S_{q1}
\end{bmatrix}
~~~~~~~~~
\vec{r}_2 = 
\begin{bmatrix} 
l_{1} C_{q1} + l_{c2}C_{q1+q2} \\
l_{1} S_{q1} + l_{c2}S_{q1+q2}
\end{bmatrix}
$$

where $C_{q1}=\cos{q_1},~S_{q1}=\sin{q_1}$

In [3]:
r1 = Matrix([[lc1*cos(q1)], 
             [lc1*sin(q1)]])

r2 = Matrix([[l1*cos(q1) + lc2*cos(q1+q2)], 
             [l1*sin(q1) + lc2*sin(q1+q2)]])

We then differentiate the displacement vectors $r_1,~r_2$ to obtain velocity vectors $v_1,~v_2$

$$
\begin{align}
&\vec{v}_1 = \frac{d}{dt}\vec{r}_1 = 
\begin{bmatrix}
-l_{c1} S_{q1} \dot{q}_1 \\
l_{c1} C_{q1} \dot{q}_1
\end{bmatrix}
\\
&\vec{v}_2 = \frac{d}{dt}\vec{r}_2 = 
\begin{bmatrix} 
-l_{1} S_{q1} \dot{q}_1 - l_{c2}S_{q1+q2}(\dot{q}_1 + \dot{q}_2) \\
 l_{1} C_{q1} \dot{q}_1 + l_{c2}C_{q1+q2}(\dot{q}_1 + \dot{q}_2)
\end{bmatrix}
\end{align}
$$

In [4]:
v1 = diff(r1, t)
simplify(v1)

Matrix([
[-l_c1*sin(q_1(t))*Derivative(q_1(t), t)],
[ l_c1*cos(q_1(t))*Derivative(q_1(t), t)]])

In [5]:
v2 = diff(r2, t)
simplify(v2)

Matrix([
[-l_1*sin(q_1(t))*Derivative(q_1(t), t) - l_c2*(Derivative(q_1(t), t) + Derivative(q_2(t), t))*sin(q_1(t) + q_2(t))],
[ l_1*cos(q_1(t))*Derivative(q_1(t), t) + l_c2*(Derivative(q_1(t), t) + Derivative(q_2(t), t))*cos(q_1(t) + q_2(t))]])

### Find the Lagrangian $\mathcal{L}$

We first obtain the translational velocity and the rotational velocity. The translational velocity is simply $v1,~v2$ we obtained before.


The rotational velocity of Acrobot is given by

$$
\begin{align}
&\omega_1 = \dot{q}_1
\\
&\omega_2 = \dot{q}_1 + \dot{q}_2
\end{align}
$$

In [6]:
w1 = dq1
w2 = dq1 + dq2

Then calculate the kinematic energy $K_e$, which is kinetic energy + rotational energy(總動能=移動動能+旋轉動能)

$$K_e = \frac{1}{2} m v^2 + \frac{1}{2} I \omega^2$$

In [7]:
from sympy.physics.vector.functions import dot

# kinematic energy
Ke = m1*v1.T.dot(v1)/2 + m2*v2.T.dot(v2)/2 + I1*w1**2/2 + I2*w2**2/2
simplify(Ke)

I_1*Derivative(q_1(t), t)**2/2 + I_2*(Derivative(q_1(t), t) + Derivative(q_2(t), t))**2/2 + l_c1**2*m_1*Derivative(q_1(t), t)**2/2 + m_2*(l_1**2*Derivative(q_1(t), t)**2 + 2*l_1*l_c2*cos(q_2(t))*Derivative(q_1(t), t)**2 + 2*l_1*l_c2*cos(q_2(t))*Derivative(q_1(t), t)*Derivative(q_2(t), t) + l_c2**2*Derivative(q_1(t), t)**2 + 2*l_c2**2*Derivative(q_1(t), t)*Derivative(q_2(t), t) + l_c2**2*Derivative(q_2(t), t)**2)/2

Then calculate the potential energy of the system

In [8]:
# potential energy
Pe = m1*g*lc1*sin(q1) + m2*g*( l1*sin(q1)+lc2*sin(q1+q2) )
simplify(Pe)

g*(l_c1*m_1*sin(q_1(t)) + m_2*(l_1*sin(q_1(t)) + l_c2*sin(q_1(t) + q_2(t))))

Finally obtain the Lagrangian

In [9]:
# Lagrangian
Lang = Ke - Pe

### Calculate Acrobot's Equations of Motion

There are several method in Sympy to obtain equations of motion(EoM) of a holonomic system.

#### 1. Calculate Lagrange EoM directly
Just directly calculate $$ \frac{d}{dt} \frac{\partial \mathcal{L}}{\partial \dot{q}} - \frac{\partial \mathcal{L}}{\partial q} =0$$

#### 2. Use Sympy built-in function "LagrangesMethod"

#### 3. Use Sympy built-in function "KanesMethod"

#### Method 1

In [10]:
# Method 1

# Calculate equations
eqq1 = simplify(diff(diff(Lang, dq1), t) - diff(Lang, q1))
eqq2 = simplify(diff(diff(Lang, dq2), t) - diff(Lang, q2))

# Obtain mass matrix
Mass_11 = simplify(diff(eqq1, ddq1))
Mass_12 = simplify(diff(eqq1, ddq2))
Mass_21 = simplify(diff(eqq2, ddq1))
Mass_22 = simplify(diff(eqq2, ddq2))
MassMatrix = Matrix([ [Mass_11, Mass_12], [Mass_21, Mass_22] ])

simplify(MassMatrix)

Matrix([
[I_1 + I_2 + l_c1**2*m_1 + m_2*(l_1**2 + 2*l_1*l_c2*cos(q_2(t)) + l_c2**2), I_2 + l_c2*m_2*(l_1*cos(q_2(t)) + l_c2)],
[                             I_2 + l_1*l_c2*m_2*cos(q_2(t)) + l_c2**2*m_2,                       I_2 + l_c2**2*m_2]])

In [11]:
Force_tmp = Matrix([[eqq1 - Mass_11*ddq1 - Mass_12*ddq2], [eqq2 - Mass_21*ddq1 - Mass_22*ddq2]])
ForceMatrix = simplify(Force_tmp - diff(Force_tmp, g)*g)
ForceMatrix

Matrix([
[-l_1*l_c2*m_2*(2*Derivative(q_1(t), t) + Derivative(q_2(t), t))*sin(q_2(t))*Derivative(q_2(t), t)],
[                                                l_1*l_c2*m_2*sin(q_2(t))*Derivative(q_1(t), t)**2]])

In [12]:
GravityMatrix = simplify( diff(Force_tmp, g)*g )
GravityMatrix

Matrix([
[g*(l_c1*m_1*cos(q_1(t)) + m_2*(l_1*cos(q_1(t)) + l_c2*cos(q_1(t) + q_2(t))))],
[                                             g*l_c2*m_2*cos(q_1(t) + q_2(t))]])

Recall the Acrobot EoM


$$
\begin{align}
&\begin{bmatrix} 
m_1 l_{c1}^2 + m_2(l_1^2 + l_{c2}^2 + 2 l_1 l_{c2} C_{q2}) + I_1 + I_2  
&& 
m_2( l_{c2}^2 + l_1 l_{c2} C_{q2} + I_2)
\\
m_2( l_{c2}^2 + l_1 l_{c2} C_{q2} + I_2)
&&
m_2 l_{c2}^2 + I_2
\end{bmatrix}
\begin{bmatrix}
\ddot{q}_1 \\
\ddot{q}_2
\end{bmatrix}
\\
+
&\begin{bmatrix}
-m_2 l_1 l_{c2} S_{q2} \dot{q}_2^2 - 2 m_1 l_1 l_{c2} S_{q2} \dot{q}_1 \dot{q}_2
\\
m_2 l_1 l_{c2} S_{q2} \dot{q}_1^2
\end{bmatrix}
+
\begin{bmatrix}
(m_1 l_{c1} + m_2 l_1)g C_{q1} + m_2 l_{c2} g C_{q1+q2}
\\
m_2 l_{c2}gC_{q1 + q2}
\end{bmatrix}
=
\begin{bmatrix}
\tau_1 
\\
\tau_2
\end{bmatrix}
\end{align}
$$

#### Method 2

In [13]:
from sympy.physics.mechanics import LagrangesMethod

LM = LagrangesMethod(Lang, [q1, q2])

The Sympy built-in function LagrangesMethod calculate the EoM. We can check it by comparing it to eqq1, eqq2.

In [14]:
LM_Lagrange_eq = simplify(LM.form_lagranges_equations())
LM_Lagrange_eq

Matrix([
[I_1*Derivative(q_1(t), (t, 2)) + I_2*(Derivative(q_1(t), (t, 2)) + Derivative(q_2(t), (t, 2))) + g*l_c1*m_1*cos(q_1(t)) + g*m_2*(l_1*cos(q_1(t)) + l_c2*cos(q_1(t) + q_2(t))) + l_c1**2*m_1*Derivative(q_1(t), (t, 2)) + m_2*(l_1**2*Derivative(q_1(t), (t, 2)) - 2*l_1*l_c2*sin(q_2(t))*Derivative(q_1(t), t)*Derivative(q_2(t), t) - l_1*l_c2*sin(q_2(t))*Derivative(q_2(t), t)**2 + 2*l_1*l_c2*cos(q_2(t))*Derivative(q_1(t), (t, 2)) + l_1*l_c2*cos(q_2(t))*Derivative(q_2(t), (t, 2)) + l_c2**2*Derivative(q_1(t), (t, 2)) + l_c2**2*Derivative(q_2(t), (t, 2)))],
[                                                                                                                                                                                                                                                                        I_2*Derivative(q_1(t), (t, 2)) + I_2*Derivative(q_2(t), (t, 2)) + g*l_c2*m_2*cos(q_1(t) + q_2(t)) + l_1*l_c2*m_2*sin(q_2(t))*Derivative(q_1(t), t)**2 + l_1*l_c2*m_2*cos(q_2(

The EoM obtained by LagrangesMethod is equal to our previous result.

In [15]:
simplify(LM_Lagrange_eq[0] - eqq1) # [0] means first equation

0

In [16]:
simplify(LM_Lagrange_eq[1] - eqq2)

0

We then obtain the mass matrix, force matrix, gravity matrix

#### Be really careful that sympy's LagrangesMethod calculate the equations by

$$\text{LM.mass_matrix} \cdot \ddot{q} = \text{LM.forcing}$$

The EoM in this article is applied by the form

$$M(q)\ddot{q} + F(q, \dot{q}) + G(q) = \tau$$

Therefore we know $F(q, \dot{q}) + G(q)$ = -1*LM.forcing

#### Be really careful the negative in LM.forcing

In [17]:
LM_MassMatrix = simplify(LM.mass_matrix);
LM_MassMatrix

Matrix([
[I_1 + I_2 + l_c1**2*m_1 + m_2*(l_1**2 + 2*l_1*l_c2*cos(q_2(t)) + l_c2**2), I_2 + l_c2*m_2*(l_1*cos(q_2(t)) + l_c2)],
[                                  I_2 + l_c2*m_2*(l_1*cos(q_2(t)) + l_c2),                       I_2 + l_c2**2*m_2]])

In [18]:
LM_TotalForce = -LM.forcing
LM_ForceMatrix = simplify(LM_TotalForce - diff(LM_TotalForce, g)*g)
LM_ForceMatrix

Matrix([
[-l_1*l_c2*m_2*(2*Derivative(q_1(t), t) + Derivative(q_2(t), t))*sin(q_2(t))*Derivative(q_2(t), t)],
[                                                l_1*l_c2*m_2*sin(q_2(t))*Derivative(q_1(t), t)**2]])

In [19]:
LM_GravityMatrix = simplify(diff(LM_TotalForce,g)*g)
LM_GravityMatrix

Matrix([
[g*(l_c1*m_1*cos(q_1(t)) + m_2*(l_1*cos(q_1(t)) + l_c2*cos(q_1(t) + q_2(t))))],
[                                             g*l_c2*m_2*cos(q_1(t) + q_2(t))]])

Recall the Acrobot EoM


$$
\begin{align}
&\begin{bmatrix} 
m_1 l_{c1}^2 + m_2(l_1^2 + l_{c2}^2 + 2 l_1 l_{c2} C_{q2}) + I_1 + I_2  
&& 
m_2( l_{c2}^2 + l_1 l_{c2} C_{q2} + I_2)
\\
m_2( l_{c2}^2 + l_1 l_{c2} C_{q2} + I_2)
&&
m_2 l_{c2}^2 + I_2
\end{bmatrix}
\begin{bmatrix}
\ddot{q}_1 \\
\ddot{q}_2
\end{bmatrix}
\\
+
&\begin{bmatrix}
-m_2 l_1 l_{c2} S_{q2} \dot{q}_2^2 - 2 m_1 l_1 l_{c2} S_{q2} \dot{q}_1 \dot{q}_2
\\
m_2 l_1 l_{c2} S_{q2} \dot{q}_1^2
\end{bmatrix}
+
\begin{bmatrix}
(m_1 l_{c1} + m_2 l_1)g C_{q1} + m_2 l_{c2} g C_{q1+q2}
\\
m_2 l_{c2}gC_{q1 + q2}
\end{bmatrix}
=
\begin{bmatrix}
\tau_1 
\\
\tau_2
\end{bmatrix}
\end{align}
$$

### Convert Sympy result to Matlab acceptable code

In [20]:
q1t, q2t = symbols('q1t q2t') # define new symbols to replace the dynamicalsymbol q(t)

MassMatrix_Matlab = LM_MassMatrix.replace(q1, q1t)
MassMatrix_Matlab = MassMatrix_Matlab.replace(q2, q2t)

#### Convert Code

In [21]:
from sympy import octave_code

octave_code(MassMatrix_Matlab[0,0])

'I_1 + I_2 + l_c1.^2.*m_1 + m_2.*(l_1.^2 + 2*l_1.*l_c2.*cos(q2t) + l_c2.^2)'

In [22]:
octave_code(MassMatrix_Matlab[0,1])

'I_2 + l_c2.*m_2.*(l_1.*cos(q2t) + l_c2)'

In [23]:
octave_code(MassMatrix_Matlab[1,0])

'I_2 + l_c2.*m_2.*(l_1.*cos(q2t) + l_c2)'

In [24]:
octave_code(MassMatrix_Matlab[1,1])

'I_2 + l_c2.^2.*m_2'

#### Convert "total" force matrix

In [25]:
dq1t, dq2t = symbols('dq1t dq2t')


from sympy import Derivative

LM_TotalForce = simplify(LM_TotalForce)

ForceMatrix_Matlab = LM_TotalForce.replace(q1, q1t)
ForceMatrix_Matlab = ForceMatrix_Matlab.replace(q2, q2t)
ForceMatrix_Matlab = ForceMatrix_Matlab.replace(Derivative(q1t,t), dq1t)
ForceMatrix_Matlab = ForceMatrix_Matlab.replace(Derivative(q2t,t), dq2t)

In [26]:
LM_TotalForce

Matrix([
[g*l_c1*m_1*cos(q_1(t)) + g*m_2*(l_1*cos(q_1(t)) + l_c2*cos(q_1(t) + q_2(t))) - l_1*l_c2*m_2*(2*Derivative(q_1(t), t) + Derivative(q_2(t), t))*sin(q_2(t))*Derivative(q_2(t), t)],
[                                                                                                   l_c2*m_2*(g*cos(q_1(t) + q_2(t)) + l_1*sin(q_2(t))*Derivative(q_1(t), t)**2)]])

In [27]:
ForceMatrix_Matlab

Matrix([
[-dq2t*l_1*l_c2*m_2*(2*dq1t + dq2t)*sin(q2t) + g*l_c1*m_1*cos(q1t) + g*m_2*(l_1*cos(q1t) + l_c2*cos(q1t + q2t))],
[                                                            l_c2*m_2*(dq1t**2*l_1*sin(q2t) + g*cos(q1t + q2t))]])

In [28]:
octave_code(ForceMatrix_Matlab[0])

'-dq2t.*l_1.*l_c2.*m_2.*(2*dq1t + dq2t).*sin(q2t) + g.*l_c1.*m_1.*cos(q1t) + g.*m_2.*(l_1.*cos(q1t) + l_c2.*cos(q1t + q2t))'

In [29]:
octave_code(ForceMatrix_Matlab[1])

'l_c2.*m_2.*(dq1t.^2.*l_1.*sin(q2t) + g.*cos(q1t + q2t))'